#### 0. Create 3 kernals, 2 images with 3 channels


In [87]:
import random as rd
import numpy as np
import struct
import math

KERNAL_SIZE = 3
IMG_SIZE    = 4

rd.seed(1234)



kernal0  = np.array([[rd.randint(1,3) for i in range(KERNAL_SIZE)] for j in range(KERNAL_SIZE)], dtype="float32")
kernal1  = np.array([[rd.randint(1,3) for i in range(KERNAL_SIZE)] for j in range(KERNAL_SIZE)], dtype="float32")
kernal2  = np.array([[rd.randint(1,3) for i in range(KERNAL_SIZE)] for j in range(KERNAL_SIZE)], dtype="float32")

img0_0    = np.array([[rd.randint(1,3) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")
img0_1    = np.array([[rd.randint(1,3) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")
img0_2    = np.array([[rd.randint(1,3) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")

img1_0    = np.array([[rd.randint(1,3) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")
img1_1    = np.array([[rd.randint(1,3) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")
img1_2    = np.array([[rd.randint(1,3) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")

weight    = np.array([[rd.randint(1,3) for i in range(2)] for j in range(2)], dtype="float32")


In [88]:
kernal0,kernal1,kernal2

(array([[2., 1., 1.],
        [1., 3., 1.],
        [3., 3., 1.]], dtype=float32),
 array([[1., 2., 1.],
        [1., 1., 1.],
        [2., 3., 3.]], dtype=float32),
 array([[2., 3., 2.],
        [1., 1., 1.],
        [3., 1., 1.]], dtype=float32))

In [89]:
img0_0,img1_0

(array([[3., 2., 1., 1.],
        [3., 3., 2., 1.],
        [3., 3., 1., 3.],
        [3., 1., 2., 3.]], dtype=float32),
 array([[1., 2., 1., 3.],
        [1., 3., 2., 2.],
        [1., 2., 2., 3.],
        [1., 1., 2., 1.]], dtype=float32))

In [90]:
weight

array([[2., 3.],
       [1., 2.]], dtype=float32)

#### 1. Do zero-padding and convolution related to signals


#### 0.Zero-padding, since reading into an array with raster scan order, place it into the correct position

In [91]:
zero_padded_img = np.zeros((IMG_SIZE+2,IMG_SIZE+2),dtype= 'float32')

In [92]:
zero_padded_img

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

In [93]:

for i in range(IMG_SIZE):
    for j in range(IMG_SIZE):
        zero_padded_img[i+1,j+1] = img0_0[i,j]


In [94]:
zero_padded_img

array([[0., 0., 0., 0., 0., 0.],
       [0., 3., 2., 1., 1., 0.],
       [0., 3., 3., 2., 1., 0.],
       [0., 3., 3., 1., 3., 0.],
       [0., 3., 1., 2., 3., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

## Replication

In [95]:
replicated_img = np.zeros((IMG_SIZE+2,IMG_SIZE+2),dtype= 'float32')
for i in range(IMG_SIZE):
    for j in range(IMG_SIZE):
        pixel = img0_0[i,j]
        # Check 4 corners
        # 4 corners
        if i == 0 and j == 0:
            replicated_img[0][0] = pixel
            replicated_img[0][1] = pixel
            replicated_img[1][0] = pixel
            replicated_img[1][1] = pixel
        elif i == 0 and j == IMG_SIZE-1:
            replicated_img[1][4] = pixel
            replicated_img[1][5] = pixel
            replicated_img[0][4] = pixel
            replicated_img[0][5] = pixel
        elif i == IMG_SIZE -1 and j == 0:
            replicated_img[4][1] = pixel
            replicated_img[4][0] = pixel
            replicated_img[5][0] = pixel
            replicated_img[5][1] = pixel
        elif i == IMG_SIZE -1 and j == IMG_SIZE-1:
            replicated_img[4][4] = pixel
            replicated_img[4][5] = pixel
            replicated_img[5][4] = pixel
            replicated_img[5][5] = pixel
        elif i == 0:
            replicated_img[0][j+1] = pixel
            replicated_img[1][j+1] = pixel
        elif j == 0:
            replicated_img[i+1][0] = pixel
            replicated_img[i+1][1] = pixel
        elif i == IMG_SIZE -1:
            replicated_img[4][j+1] = pixel
            replicated_img[5][j+1] = pixel
        elif j == IMG_SIZE -1:
            replicated_img[i+1][4] = pixel
            replicated_img[i+1][5] = pixel
        else:
            replicated_img[i+1][j+1] = pixel


In [96]:
replicated_img

array([[3., 3., 2., 1., 1., 1.],
       [3., 3., 2., 1., 1., 1.],
       [3., 3., 3., 2., 1., 1.],
       [3., 3., 3., 1., 3., 3.],
       [3., 3., 1., 2., 3., 3.],
       [3., 3., 1., 2., 3., 3.]], dtype=float32)

In [97]:
def zero_padded_img(img):
    zero_padded_img = np.zeros((IMG_SIZE+2,IMG_SIZE+2),dtype= 'float32')
    for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            zero_padded_img[i+1,j+1] = img[i,j]

    return zero_padded_img

def replication(img):
    replicated_img = np.zeros((IMG_SIZE+2,IMG_SIZE+2),dtype= 'float32')
    for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            pixel = img[i,j]
            # Check 4 corners
            # 4 corners
            if i == 0 and j == 0:
                replicated_img[0][0] = pixel
                replicated_img[0][1] = pixel
                replicated_img[1][0] = pixel
                replicated_img[1][1] = pixel
            elif i == 0 and j == IMG_SIZE-1:
                replicated_img[1][4] = pixel
                replicated_img[1][5] = pixel
                replicated_img[0][4] = pixel
                replicated_img[0][5] = pixel
            elif i == IMG_SIZE -1 and j == 0:
                replicated_img[4][1] = pixel
                replicated_img[4][0] = pixel
                replicated_img[5][0] = pixel
                replicated_img[5][1] = pixel
            elif i == IMG_SIZE -1 and j == IMG_SIZE-1:
                replicated_img[4][4] = pixel
                replicated_img[4][5] = pixel
                replicated_img[5][4] = pixel
                replicated_img[5][5] = pixel
            elif i == 0:
                replicated_img[0][j+1] = pixel
                replicated_img[1][j+1] = pixel
            elif j == 0:
                replicated_img[i+1][0] = pixel
                replicated_img[i+1][1] = pixel
            elif i == IMG_SIZE -1:
                replicated_img[4][j+1] = pixel
                replicated_img[5][j+1] = pixel
            elif j == IMG_SIZE -1:
                replicated_img[i+1][4] = pixel
                replicated_img[i+1][5] = pixel
            else:
                replicated_img[i+1][j+1] = pixel

    return replicated_img


### Use Zeropadded for further testing

In [98]:
img0_0 = zero_padded_img(img0_0)
img0_1 = zero_padded_img(img0_1)
img0_2 = zero_padded_img(img0_2)

In [99]:
img0_0

array([[0., 0., 0., 0., 0., 0.],
       [0., 3., 2., 1., 1., 0.],
       [0., 3., 3., 2., 1., 0.],
       [0., 3., 3., 1., 3., 0.],
       [0., 3., 1., 2., 3., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

#### 2. Sum add convolution, fill up 4 max pooling results.


In [100]:
convoluted_results = np.zeros((IMG_SIZE,IMG_SIZE),dtype= 'float32')

In [101]:
for i in range(IMG_SIZE):
    for j in range(IMG_SIZE):
        conv1,conv2,conv3 = np.float32(0),np.float32(0),np.float32(0)
        for k in range(KERNAL_SIZE):
            for l in range(KERNAL_SIZE):
                conv1 += kernal0[k,l] * img0_0[i+k,j+l]
                conv2 += kernal1[k,l] * img0_1[i+k,j+l]
                conv3 += kernal2[k,l] * img0_2[i+k,j+l]

        convoluted_results[i,j] = conv1 + conv2 + conv3


In [102]:
def convolution(img0_0, img0_1,img0_2):
    convoluted_results = np.zeros((IMG_SIZE,IMG_SIZE),dtype= 'float32')
    for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            conv1,conv2,conv3 = np.float32(0),np.float32(0),np.float32(0)
            for k in range(KERNAL_SIZE):
                for l in range(KERNAL_SIZE):
                    conv1 += kernal0[k,l] * img0_0[i+k,j+l]
                    conv2 += kernal1[k,l] * img0_1[i+k,j+l]
                    conv3 += kernal2[k,l] * img0_2[i+k,j+l]

            convoluted_results[i,j] = conv1 + conv2 + conv3

    return convoluted_results

In [103]:
type(conv1)

numpy.float32

In [104]:
convoluted_results

array([[ 47.,  67.,  59.,  37.],
       [ 64., 100.,  99.,  66.],
       [ 70.,  92.,  83.,  60.],
       [ 37.,  51.,  55.,  36.]], dtype=float32)

### Max pooling

In [105]:
max_pooled_result = np.zeros((2,2),dtype='float32')
for i in range(0,4,2):
    for j in range(0,4,2):
        temp_max = np.float32(0)
        for k in range(0,2):
            for l in range(0,2):
                if temp_max < convoluted_results[i+k,j+l]:
                    temp_max = convoluted_results[i+k,j+l]

        max_pooled_result[i//2,j//2] = temp_max


In [106]:
def max_pooling(convoluted_results):
    max_pooled_result = np.zeros((2,2),dtype='float32')
    # Finish this with 2 cycles
    for i in range(0,4,2):
        # Parallelize this using 2 comparators
        for j in range(0,4,2):
            temp_max = np.float32(0)
            for k in range(0,2):
                for l in range(0,2):
                    if temp_max < convoluted_results[i+k,j+l]:
                        temp_max = convoluted_results[i+k,j+l]

            max_pooled_result[i//2,j//2] = temp_max

    return max_pooled_result


In [107]:
type(temp_max)

numpy.float32

In [108]:
max_pooled_result

array([[100.,  99.],
       [ 92.,  83.]], dtype=float32)

In [109]:
weight

array([[2., 3.],
       [1., 2.]], dtype=float32)

#### 3. Fully connected layers with matrix multiplication
### Finding the min-max of fully as computing the fully connected layers.

In [110]:
fully_connected_result = np.zeros((4,1),dtype='float32')

LARGE_NUMBER = 99999999999999999999999999999
SMALL_NUMBER = -99999999999999999999999999999

min = np.float32(LARGE_NUMBER)
max = np.float32(SMALL_NUMBER)

for i in range(2):
    for j in range(2):
        partial_mult = 0
        for k in range(2):
            partial_mult += max_pooled_result[i,k] * weight[k,j]

        if partial_mult > max:
            max = partial_mult
        if partial_mult < min:
            min = partial_mult

        fully_connected_result[i*2+j] = partial_mult


In [111]:
def fully_connection(max_pooled_result,weight):
    fully_connected_result = np.zeros((4,1),dtype='float32')
    min = np.float32(LARGE_NUMBER)
    max = np.float32(SMALL_NUMBER)

    for i in range(2):
        for j in range(2):
            partial_mult = 0
            for k in range(2):
                partial_mult += max_pooled_result[i,k] * weight[k,j]

            if partial_mult > max:
                max = partial_mult
            if partial_mult < min:
                min = partial_mult

            fully_connected_result[i*2+j] = partial_mult

    return fully_connected_result


In [112]:
type(min)

numpy.float64

In [113]:
fully_connected_result = fully_connection(max_pooled_result,weight)

In [114]:
fully_connected_result

array([[299.],
       [498.],
       [267.],
       [442.]], dtype=float32)

In [115]:
type(max),min

(numpy.float64, 267.0)

#### 5. Do the min-max Normalization


In [116]:
x_scaled = np.zeros((4,1),dtype='float32')
for i,x in enumerate(fully_connected_result):
    x_scaled[i] = (x-min) / (max-min)

In [117]:
def min_max_norm(fully_connected_result):
    x_scaled = np.zeros((4,1),dtype='float32')
    for i,x in enumerate(fully_connected_result):
        x_scaled[i] = (x-min) / (max-min)

    return x_scaled

In [118]:
x_scaled = min_max_norm(fully_connected_result)

In [119]:
x_scaled

array([[0.13852814],
       [1.        ],
       [0.        ],
       [0.75757575]], dtype=float32)

#### 6. Do the sigmoid activation function

## Sigmoid

In [120]:
activated_value = np.zeros((4,1),dtype='float32')

In [121]:
for i,e in enumerate(x_scaled):
    activated_value[i] = 1/(1+np.exp(-e,dtype='float32'))

In [122]:
def sigmoid(x_scaled):
    activated_value = np.zeros((4,1),dtype='float32')
    for i,e in enumerate(x_scaled):
        activated_value[i] = 1/(1+np.exp(-e,dtype='float32'))

    return activated_value

In [123]:
activated_value

array([[0.5345768],
       [0.7310586],
       [0.5      ],
       [0.6808272]], dtype=float32)

## Tanh

In [124]:
for i,e in enumerate(x_scaled):
    activated_value[i] = (np.exp(e,dtype='float32') - np.exp(-e,dtype='float32')) \
    / (np.exp(e,dtype='float32') + np.exp(-e,dtype='float32'))

In [125]:
def tanh(x_scaled):
    activated_value = np.zeros((4,1),dtype='float32')
    for i,e in enumerate(x_scaled):
        activated_value[i] = 1/(1+np.exp(-e,dtype='float32'))

    return activated_value

## Hex representaion conversion

In [126]:
activated_value

array([[0.13764876],
       [0.7615942 ],
       [0.        ],
       [0.6396468 ]], dtype=float32)

In [127]:
activated_value[0][0]

0.13764876

In [128]:
value = activated_value[0][0]
# Get the hexadecimal representation (as a string)
float_as_int = np.float32(value).view(np.int32)
hex_representation = format(value.view(np.int32), 'x')

In [129]:
hex_representation

'3e0cf3cc'